# Miniproject 1: Image Classification

## Introduction

### Description

One of the deepest traditions in learning about deep learning is to first [tackle the exciting problem of MNIST classification](http://deeplearning.net/tutorial/logreg.html). [The MNIST database](https://en.wikipedia.org/wiki/MNIST_database) (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that was [recently extended](https://arxiv.org/abs/1702.05373). We break with this tradition (just a little bit) and tackle first the related problem of classifying cropped, downsampled and grayscaled images of house numbers in the [The Street View House Numbers (SVHN) Dataset](http://ufldl.stanford.edu/housenumbers/).


### Prerequisites

- You should have a running installation of [tensorflow](https://www.tensorflow.org/install/) and [keras](https://keras.io/).
- You should know the concepts "multilayer perceptron", "stochastic gradient descent with minibatches", "training and validation data", "overfitting" and "early stopping".

### What you will learn

- You will learn how to define feedforward neural networks in keras and fit them to data.
- You will be guided through a prototyping procedure for the application of deep learning to a specific domain.
- You will get in contact with concepts discussed later in the lecture, like "regularization", "batch normalization" and "convolutional networks".
- You will gain some experience on the influence of network architecture, optimizer and regularization choices on the goodness of fit.
- You will learn to be more patient :) Some fits may take your computer quite a bit of time; run them over night.

### Evaluation criteria

The evaluation is (mostly) based on the figures you submit and your answer sentences. 
We will only do random tests of your code and not re-run the full notebook.

### Your names

Before you start, please enter your full name(s) in the field below; they are used to load the data. The variable student2 may remain empty, if you work alone.

In [ ]:
student1 = 233984
student2 = 285467

## Some helper functions

For your convenience we provide here some functions to preprocess the data and plot the results later. Simply run the following cells with `Shift-Enter`.

### Dependencies and constants

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy.io

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import SGD, Adam

# you may experiment with different subsets, 
# but make sure in the submission 
# it is generated with the correct random seed for all exercises.
np.random.seed(hash(student1 + student2) % 2**32)
subset_of_classes = np.random.choice(range(10), 5, replace = False)

In [ ]:
np.random.choice(range(10), 5, replace = False) #random choice of subclasses

### Plotting

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
def plot_some_samples(x, y = [], yhat = [], select_from = [], 
                      ncols = 6, nrows = 4, xdim = 16, ydim = 16,
                      label_mapping = range(10)):
    """plot some input vectors as grayscale images (optionally together with their assigned or predicted labels).
    
    x is an NxD - dimensional array, where D is the length of an input vector and N is the number of samples.
    Out of the N samples, ncols x nrows indices are randomly selected from the list select_from (if it is empty, select_from becomes range(N)).
    
    Keyword arguments:
    y             -- corresponding labels to plot in green below each image.
    yhat          -- corresponding predicted labels to plot in red below each image.
    select_from   -- list of indices from which to select the images.
    ncols, nrows  -- number of columns and rows to plot.
    xdim, ydim    -- number of pixels of the images in x- and y-direction.
    label_mapping -- map labels to digits.
    
    """
    fig, ax = plt.subplots(nrows, ncols)
    if len(select_from) == 0:
        select_from = range(x.shape[0])
    indices = np.random.choice(select_from, size = min(ncols * nrows, len(select_from)), replace = False)
    for i, ind in enumerate(indices):
        thisax = ax[i//ncols,i%ncols]
        thisax.matshow(x[ind].reshape(xdim, ydim), cmap='gray')
        thisax.set_axis_off()
        if len(y) != 0:
            j = y[ind] if type(y[ind]) != np.ndarray else y[ind].argmax()
            thisax.text(0, 0, (label_mapping[j]+1)%10, color='green', 
                                                       verticalalignment='top',
                                                       transform=thisax.transAxes)
        if len(yhat) != 0:
            k = yhat[ind] if type(yhat[ind]) != np.ndarray else yhat[ind].argmax()
            thisax.text(1, 0, (label_mapping[k]+1)%10, color='red',
                                             verticalalignment='top',
                                             horizontalalignment='right',
                                             transform=thisax.transAxes)
    return fig

def prepare_standardplot(title, xlabel):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    ax1.set_ylabel('categorical cross entropy')
    ax1.set_xlabel(xlabel)
    ax1.set_yscale('log')
    ax2.set_ylabel('accuracy [% correct]')
    ax2.set_xlabel(xlabel)
    return fig, ax1, ax2

def finalize_standardplot(fig, ax1, ax2):
    ax1handles, ax1labels = ax1.get_legend_handles_labels()
    if len(ax1labels) > 0:
        ax1.legend(ax1handles, ax1labels)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels)
    fig.tight_layout()
    plt.subplots_adjust(top=0.9)

def plot_history(history, title):
    fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
    ax1.plot(history.history['loss'], label = "training")
    ax1.plot(history.history['val_loss'], label = "validation")
    ax2.plot(history.history['acc'], label = "training")
    ax2.plot(history.history['val_acc'], label = "validation")
    finalize_standardplot(fig, ax1, ax2)
    return fig

### Loading and preprocessing the data

The data consists of RGB color images with 32x32 pixels, loaded into an array of dimension 32x32x3x(number of images). We convert them to grayscale (using [this method](https://en.wikipedia.org/wiki/SRGB#The_reverse_transformation)) and we downsample them to images of 16x16 pixels by averaging over patches of 2x2 pixels.

With these preprocessing steps we obviously remove some information that could be helpful in classifying the images. But, since the processed data is much lower dimensional, the fitting procedures converge faster. This is an advantage in situations like here (or generally when prototyping), were we want to try many different things without having to wait too long for computations to finish. After having gained some experience, one may want to go back to work on the 32x32 RGB images.


In [ ]:
# convert RGB images x to grayscale using the formula for Y_linear in https://en.wikipedia.org/wiki/Grayscale#Colorimetric_(perceptual_luminance-preserving)_conversion_to_grayscale
def grayscale(x):
    x = x.astype('float32')/255
    x = np.piecewise(x, [x <= 0.04045, x > 0.04045], 
                        [lambda x: x/12.92, lambda x: ((x + .055)/1.055)**2.4])
    return .2126 * x[:,:,0,:] + .7152 * x[:,:,1,:]  + .07152 * x[:,:,2,:]

def downsample(x):
    return sum([x[i::2,j::2,:] for i in range(2) for j in range(2)])/4

def preprocess(data):
    gray = grayscale(data['X'])
    downsampled = downsample(gray)
    print(downsampled[0][0])
    return (downsampled.reshape(16*16, gray.shape[2]).transpose(),
            data['y'].flatten() - 1)


data_train = scipy.io.loadmat('housenumbers/train_32x32.mat')
data_test = scipy.io.loadmat('housenumbers/test_32x32.mat')

x_train_all, y_train_all = preprocess(data_train)
x_test_all, y_test_all = preprocess(data_test)

In [ ]:
x_train_all.T.reshape(x_train_all.shape[0],1, 16,16)[0][0]

### Selecting a subset of classes

We furter reduce the size of the dataset (and thus reduce computation time) by selecting only the 5 (out of 10 digits) in subset_of_classes.

In [ ]:
def extract_classes(x, y, classes):
    indices = []
    labels = []
    count = 0
    for c in classes:
        tmp = np.where(y == c)[0]
        indices.extend(tmp)
        labels.extend(np.ones(len(tmp), dtype='uint8') * count)
        count += 1
    return x[indices], labels

x_train, y_train = extract_classes(x_train_all, y_train_all, subset_of_classes)
x_test, y_test = extract_classes(x_test_all, y_test_all, subset_of_classes)

Let us plot some examples now. The green digit at the bottom left of each image indicates the corresponding label in y_test.
For further usage of the function plot_some_samples, please have a look at its definition in the plotting section.

In [ ]:
plot_some_samples(x_test, y_test, label_mapping = subset_of_classes);

To prepare for fitting we transform the labels to one hot coding, i.e. for 5 classes, label 2 becomes the vector [0, 0, 1, 0, 0] (python uses 0-indexing).

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

## Exercise 1: No hidden layer

### Description

Define and fit a model without a hidden layer. 

1. Use the softmax activation for the output layer.
2. Use the categorical_crossentropy loss.
3. Add the accuracy metric to the metrics.
4. Choose stochastic gradient descent for the optimizer.
5. Choose a minibatch size of 128.
6. Fit for as many epochs as needed to see no further decrease in the validation loss.
7. Plot the output of the fitting procedure (a history object) using the function plot_history defined above.
8. Determine the indices of all test images that are misclassified by the fitted model and plot some of them using the function 
   `plot_some_samples(x_test, y_test, yhat_test, error_indices, label_mapping = subset_of_classes)`


Hints:
* Read the keras docs, in particular [Getting started with the Keras Sequential model](https://keras.io/getting-started/sequential-model-guide/).
* Have a look at the keras [examples](https://github.com/keras-team/keras/tree/master/examples), e.g. [mnist_mlp](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py).

### Solution

In [ ]:
print(x_train.shape[0], 'number of train samples')
print(x_test.shape[0], 'number of test samples')

print(x_train.shape[1], 'input dim samples')
print(y_test.shape[1], 'output test samples')

In [ ]:
activation = 'softmax' #1. use softmax
loss = 'categorical_crossentropy' #2. use categorical_crossentropy for loss
metrics=['accuracy'] #3. add 'accuray' to metrics
optimizer = SGD() #4, SGD as optimizer
minibatch = 128 #5. define minibatch size

num_classes = y_test.shape[1]

model = Sequential()

model.add(Dense(num_classes, activation=activation, input_dim=x_train.shape[1]))

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)

In [ ]:
epochs = 50 # to be optimized

history = model.fit(x_train, y_train, batch_size=minibatch, epochs=epochs, validation_data=(x_test, y_test)) #validation split

In [ ]:
plot_history(history, "no hidden layer model")
plt.show()

In [ ]:
pred = model.predict(x_test)
indexes = np.nonzero(pred.argmax(axis=1) != y_test.argmax(axis=1))[0]

In [ ]:
plot_some_samples(x_test, y_test, pred, indexes, label_mapping = subset_of_classes)

## Exercise 2: One hidden layer, different optizimizers
### Description

Train a network with one hidden layer and compare different optimizers.

1. Use one hidden layer with 64 units and the 'relu' activation. Use the [summary method](https://keras.io/models/about-keras-models/) to inspect your model.
2. Fit the model for 50 epochs with different learning rates of stochastic gradient descent and answer the question below.
3. Replace the stochastic gradient descent optimizer with the [Adam optimizer](https://keras.io/optimizers/#adam).
4. Plot the learning curves of SGD with a reasonable learning rate together with the learning curves of Adam in the same figure. Take care of a reasonable labeling of the curves in the plot.

### Solution

In [ ]:
#part 1 and 2 of question

#general parameters
activation = 'relu'
loss = 'categorical_crossentropy'
metrics=['accuracy']
minibatch = 128 #5. define minibatch size
num_classes = y_test.shape[1]
epochs = 50
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5, 1]

histories = []

for rate in learning_rates:
    model = Sequential()
    optimizer = SGD(rate)
    model.add(Dense(64, activation=activation, input_dim=x_train.shape[1]))
    model.add(Dense(num_classes, activation='softmax', name='outputlayer'))

    model.compile(loss=loss, 
                  optimizer=optimizer,
                  metrics=metrics)
    model.summary()
    
    history = model.fit(x_train, y_train, batch_size=minibatch, epochs=epochs, 
                        validation_data=(x_test, y_test), verbose=0)
    histories.append(history)
    plot_history(history, rate)
    plt.show()

In [ ]:
#part 3 of question
model = Sequential()
optimizer = Adam()
model.add(Dense(64, activation=activation, input_dim=x_train.shape[1]))
model.add(Dense(num_classes, activation='softmax', name='outputlayer'))

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)
model.summary()

history_adam = model.fit(x_train, y_train, batch_size=minibatch, epochs=epochs, validation_data=(x_test, y_test), verbose=0)
plot_history(history_adam, rate)
plt.plot()

In [ ]:
#part 4 plot learning curves

plt.suptitle('Convergence of accuracy SGD vs Adam')

plt.subplot(1,2,1)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.title('training accuracy')
plt.plot(history_adam.history['acc'])
plt.plot(histories[-3].history['acc'])
plt.legend(['adam', 'sgd'])

plt.subplot(1,2,2)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.title('testing accuracy')
plt.plot(history_adam.history['val_acc'])
plt.plot(histories[-3].history['val_acc'])
plt.legend(['adam', 'sgd'])

**Question**: What happens if the learning rate of SGD is A) very large B) very small? Please answer A) and B) with one full sentence (double click this markdown cell to edit).

**Answer**:
What do we define as very large and very small?

A) Fast convergence but then gets stuck at valze above optimal (acc only 0.2089 for lr=10), for very large starts to diverge.

B) if too small doesn't converge anymore. 

for reasonably slow valealgorithm converges very slowly, but to lower value (0.29 for lr=0.001)

## Exercise 3: Overfitting and early stopping with Adam

### Description

Run the above simulation with Adam for sufficiently many epochs (be patient!) until you see clear overfitting.

1. Plot the learning curves of a fit with Adam and sufficiently many epochs and answer the questions below.

A simple, but effective mean to avoid overfitting is early stopping, i.e. a fit is not run until convergence but stopped as soon as the validation error starts to increase. We will use early stopping in all subsequent exercises.

### Solution

In [ ]:
activation = 'relu'
loss = 'categorical_crossentropy'
metrics=['accuracy']
optimizer = Adam() # does not converge fully
minibatch = 128 #5. define minibatch size

num_classes = y_test.shape[1]

model = Sequential()

model.add(Dense(64, activation=activation, input_dim=x_train.shape[1]))
model.add(Dense(num_classes, activation='softmax', name='outputlayer'))

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)


#issue with model size, eg loss becomes NAN
epochs = 400
history_3 = model.fit(x_train, y_train, batch_size=minibatch, epochs=epochs, validation_data=(x_test, y_test), verbose=0)

In [ ]:
plot_history(history_3, "overfitting with adam")
plt.show()

**Question 1**: At which epoch (approximately) does the model start to overfit? Please answer with one full sentence.

**Answer**: 

Ca around epoch 40

**Question 2**: Explain the qualitative difference between the loss curves and the accuracy curves with respect to signs of overfitting. Please answer with at most 3 full sentences.

**Answer**: 



## Exercise 4: Model performance as a function of number of hidden neurons

### Description

Investigate how the best validation loss and accuracy depends on the number of hidden neurons in a single layer.

1. Fit a reasonable number of models with different hidden layer size (between 10 and 1000 hidden neurons) for a fixed number of epochs well beyond the point of overfitting.
2. Collect some statistics by fitting the same models as in 1. for multiple initial conditions. Hints: 1. If you don't reset the random seed, you get different initial conditions each time you create a new model. 2. Let your computer work while you are asleep.
3. Plot summary statistics of the final validation loss and accuracy versus the number of hidden neurons. Hint: [boxplots](https://matplotlib.org/examples/pylab_examples/boxplot_demo.html) (also [here](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.boxplot.html?highlight=boxplot#matplotlib.axes.Axes.boxplot)) are useful. You may also want to use the matplotlib method set_xticklabels.
4. Plot summary statistics of the loss and accuracy for early stopping versus the number of hidden neurons.

### Solution

In [ ]:
def generate_model(layer_size=10, activation = 'relu', 
                   loss = 'categorical_crossentropy', metrics=['accuracy'],
                   optimizer = Adam(), num_classes = y_test.shape[1],
                   input_dim=x_train.shape[1]):

    model = Sequential()

    model.add(Dense(layer_size, activation=activation, input_dim=input_dim))
    model.add(Dense(num_classes, activation=activation, name='outputlayer'))

    model.compile(loss=loss, 
                  optimizer=optimizer,
                  metrics=metrics)
    return model

def fit_model(model, x_train, y_train,validation_data, minibatch = 128, epochs=50):
    history_ = model.fit(x_train, y_train, 
                          batch_size=minibatch, 
                          epochs=epochs, 
                          validation_data=validation_data, 
                          verbose=0)
    return history

In [ ]:
layer_sizes = range(10, 1000, 100)
statistics_history = []


for layer_size in layer_sizes:
    for i in range(1, 50, 1):
        model = generate_model(layer_size=layer_size)
        history = fit_model(model, x_train, y_train, (x_test, y_test))
        statistics_history.append(([history.history['acc'][-1], history.history['val_acc'][-1]],
                                   history.history['loss'][-1], history.history['val_loss'][-1],
                                   layer_size))
    
with open('layersize_stat_history_data', 'wb') as file_pi:
        pickle.dump(statistics_history, file_pi)

## Exercise 5: Comparison to deep models

### Description

Instead of choosing one hidden layer (with many neurons) you experiment here with multiple hidden layers (each with not so many neurons).

1. Fit models with 2, 3 and 4 hidden layers with approximately the same number of parameters as a network with one hidden layer of 100 neurons. Hint: Calculate the number of parameters in a network with input dimensionality N_in, K hidden layers with N_h units, one output layer with N_out dimensions and solve for N_h. Confirm you result with the keras method model.summary().
2. Run each model multiple times with different initial conditions and plot summary statistics of the best validation loss and accuracy versus the number of hidden layers.

### Solution

a network with one hidden layer of 100 neurons has

$$100*(|input|+1) + (100+1)*|output| = 100 * (256+1) + (100+1)*5 = 26,205$$

parameters.

Hence for a system with 2 hidden layers to have a similar number of parameters we should have that

$$ x*(|input|+1)+(x+1)*y + (y+1)*|output| = 26,205$$


integer solution 91, then 27
or ca 77 per layer


For 3 hidden layers we have:

$$ x*(|input|+1)+(x+1)*y +(y+1)*z+(z+1)*|output| = 26,205$$
or ca 66 per layer


For 4 we have:

$$ x*(|input|+1)+(x+1)*y + (y+1)*z + (z+1) * \alpha + (\alpha +1)*|output| = 26,205$$ 
or ca 59


# do we put the same number of neurons at each hidden layer?

In [ ]:
#singular hidden layer
activation = 'relu'
loss = 'categorical_crossentropy'
metrics=['accuracy']
minibatch = 128 #5. define minibatch size
num_classes = y_test.shape[1]
epochs = 50
optimizer = Adam()


model = Sequential()
model.add(Dense(100, activation=activation, input_dim=x_train.shape[1]))
model.add(Dense(num_classes, activation='softmax', name='outputlayer'))

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)
model.summary()

In [ ]:
histories_2 = []
histories_3 = []
histories_4 = []


#should we optimize the hyper parameters?

for i in range (0,3, 1):
    # more similar sized layers
    model_2_2 = Sequential()
    model_2_2.add(Dense(78, activation=activation, input_dim=x_train.shape[1]))
    model_2_2.add(Dense(73, activation=activation, input_dim=78))
    model_2_2.add(Dense(num_classes, activation='softmax', name='outputlayer'))

    model_2_2.compile(loss=loss, 
                  optimizer=optimizer,
                  metrics=metrics)
    
    history_ = model_2_2.fit(x_train, y_train, 
                          batch_size=minibatch, 
                          epochs=epochs, 
                          validation_data=(x_test, y_test), 
                          verbose=0)
    histories_2.append(history_.history)
    
    #3 hidden layer model 
    model_3 = Sequential()
    model_3.add(Dense(66, activation=activation, input_dim=x_train.shape[1]))
    model_3.add(Dense(66, activation=activation, input_dim=66))
    model_3.add(Dense(67, activation=activation, input_dim=66))
    model_3.add(Dense(num_classes, activation='softmax', name='outputlayer'))

    model_3.compile(loss=loss, 
                  optimizer=optimizer,
                  metrics=metrics)
    
    history_ = model_3.fit(x_train, y_train, 
                          batch_size=minibatch, 
                          epochs=epochs, 
                          validation_data=(x_test, y_test), 
                          verbose=0)
    histories_3.append(history_.history)

    # 4 hidden layer model

    model_4 = Sequential()
    model_4.add(Dense(59, activation=activation, input_dim=x_train.shape[1]))
    model_4.add(Dense(59, activation=activation, input_dim=59))
    model_4.add(Dense(59, activation=activation, input_dim=59))
    model_4.add(Dense(59, activation=activation, input_dim=59))
    model_4.add(Dense(num_classes, activation='softmax', name='outputlayer'))

    model_4.compile(loss=loss, 
                  optimizer=optimizer,
                  metrics=metrics)
    
    history_ = model_4.fit(x_train, y_train, 
                          batch_size=minibatch, 
                          epochs=epochs, 
                          validation_data=(x_test, y_test), 
                          verbose=0)
    histories_4.append(history_.history)
    
    if( i == 0):
        model_2_2.summary()
        model_3.summary()
        model_4.summary()

## Exercise 6: Tricks (regularization, batch normalization, dropout)

### Description

Overfitting can also be counteracted with regularization and dropout. Batch normalization is supposed to mainly decrease convergence time.

1. Try to improve the best validation scores of the model with 1 layer and 100 hidden neurons and the model with 4 hidden layers. Experiment with batch_normalization layers, dropout layers and l1- and l2-regularization on weights (kernels) and biases.
2. After you have found good settings, plot for both models the learning curves of the naive model you fitted in the previous exercises together with the learning curves of the current version.
3. For proper comparison, plot also the learning curves of the two current models in a third figure.

### Solution

In [ ]:
#model with 1 layer and 100 hidden neurons

In [ ]:
#model with 4 hidden layers

In [ ]:
#regularized and non regularized

## Exercise 7: Convolutional networks

### Description

Convolutional neural networks have an inductive bias that is well adapted to image classification.

1. Design a convolutional neural network, play with the parameters and fit it. Hint: You may get valuable inspiration from the keras [examples](https://github.com/keras-team/keras/tree/master/examples), e.g. [mnist_cnn](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py).
2. Plot the learning curves of the convolutional neural network together with the so far best performing model.

## Solution

In [ ]:
#reformating training samples

x_test_cnn = x_test.T.reshape( x_test.shape[0], 16,16,1)
x_train_cnn = x_train.T.reshape(x_train.shape[0], 16,16,1)

In [ ]:
x_test_cnn[0].shape

In [ ]:
#98~99% accuracy to be expected

epochs = 12
batch_size= 128
kernel_size = (3,3)
activation = 'relu'
optimizer = keras.optimizers.Adadelta()
loss='categorical_crossentropy'
img_size=16 #because of down sampling
num_classes = y_test.shape[1]

input_shape = (img_size, img_size, 1)


model = Sequential()
model.add(Conv2D(16, kernel_size=kernel_size,
                 activation=activation, input_shape=input_shape, data_format="channels_last"))

model.add(Conv2D(32, kernel_size, activation=activation))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten()) 
model.add(Dense(64, activation=activation))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

history = model.fit(x_train_cnn, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_cnn, y_test))

score = model.evaluate(x_test_cnn, y_test, verbose=0)


In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(https://github.com/yashk2810/MNIST-Keras/blob/master/Notebook/MNIST_keras_CNN-99.55%25.ipynb)

https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html

https://machinelearningmastery.com/keras-functional-api-deep-learning/